In [3]:
import os

import gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.common.policies import MlpPolicy, FeedForwardPolicy
from stable_baselines import results_plotter
from stable_baselines import *

In [38]:
import gym
import numpy as np

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Customize value network 
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           net_arch=[dict(vf=[10, 7, 5])],
                                           feature_extraction="mlp")
        
# Create and wrap the environment
env = gym.make('NChain-v0')
env = Monitor(env, log_dir, allow_early_resets=True)
env = DummyVecEnv(([lambda: env]))

model = TRPO(CustomPolicy, env, verbose=1)

# Train the agent
time_steps = 800000
model.learn(total_timesteps=time_steps)

********** Iteration 0 ************
Optimizing Policy...
sampling
done in 0.271 seconds
computegrad
done in 0.101 seconds
conjugate_gradient
      iter residual norm  soln norm
         0    0.00301          0
         1    0.00175      0.139
         2   0.000509      0.428
         3   4.21e-05      0.651
         4   1.16e-06       0.68
         5   3.83e-11      0.682
done in 0.081 seconds
Expected: 0.022 Actual: 0.021
Stepsize OK!
vf
done in 0.065 seconds
-----------------------------------------
| EpLenMean               | 1e+03       |
| EpRewMean               | 1.36e+03    |
| EpThisIter              | 1           |
| EpisodesSoFar           | 1           |
| TimeElapsed             | 0.586       |
| TimestepsSoFar          | 1024        |
| entloss                 | 0.0         |
| entropy                 | 0.68514144  |
| explained_variance_t... | 0.00297     |
| meankl                  | 0.008621283 |
| optimgain               | 0.021068435 |
| surrgain                | 0.0

Stepsize OK!
vf
done in 0.020 seconds
-----------------------------------------
| EpLenMean               | 1e+03       |
| EpRewMean               | 2.22e+03    |
| EpThisIter              | 1           |
| EpisodesSoFar           | 8           |
| TimeElapsed             | 2.49        |
| TimestepsSoFar          | 8192        |
| entloss                 | 0.0         |
| entropy                 | 0.4090485   |
| explained_variance_t... | 0.00135     |
| meankl                  | 0.009228535 |
| optimgain               | 0.005043515 |
| surrgain                | 0.005043515 |
-----------------------------------------
********** Iteration 8 ************
Optimizing Policy...
sampling
done in 0.235 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   0.000268          0
         1   2.49e-05     0.0426
         2   1.95e-05      0.107
         3   9.17e-07      0.272
         4   1.06e-09      0.295
         5   2.31e-09      0.295

********** Iteration 15 ************
Optimizing Policy...
sampling
done in 0.232 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   0.000598          0
         1   1.26e-06     0.0852
         2    5.7e-07     0.0967
         3   2.19e-10      0.108
         4   5.85e-12      0.108
done in 0.003 seconds
Expected: 0.007 Actual: 0.006
Stepsize OK!
vf
done in 0.018 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 2.69e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 16           |
| TimeElapsed             | 4.64         |
| TimestepsSoFar          | 16384        |
| entloss                 | 0.0          |
| entropy                 | 0.21219334   |
| explained_variance_t... | 0.0013       |
| meankl                  | 0.008639605  |
| optimgain               | 0.0059701256 |
| surrgain                | 0.0059701256 |
--------

********** Iteration 23 ************
Optimizing Policy...
sampling
done in 0.228 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   4.19e-06          0
         1   2.72e-06     0.0351
         2   2.58e-11     0.0886
done in 0.003 seconds
Expected: 0.002 Actual: 0.001
Stepsize OK!
vf
done in 0.017 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 2.97e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 24           |
| TimeElapsed             | 6.74         |
| TimestepsSoFar          | 24576        |
| entloss                 | 0.0          |
| entropy                 | 0.10296066   |
| explained_variance_t... | 0.000626     |
| meankl                  | 0.0043543954 |
| optimgain               | 0.001169622  |
| surrgain                | 0.001169622  |
------------------------------------------
********** Iteration 24 *******

********** Iteration 31 ************
Optimizing Policy...
sampling
done in 0.237 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   9.17e-08          0
         1   2.58e-12     0.0253
done in 0.001 seconds
Expected: 0.000 Actual: 0.002
Stepsize OK!
vf
done in 0.015 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 3.14e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 32           |
| TimeElapsed             | 8.81         |
| TimestepsSoFar          | 32768        |
| entloss                 | 0.0          |
| entropy                 | 0.030696953  |
| explained_variance_t... | 0.000337     |
| meankl                  | 0.0041829245 |
| optimgain               | 0.0015663161 |
| surrgain                | 0.0015663161 |
------------------------------------------
********** Iteration 32 ************
Optimizing Policy...
sampli

...

Stepsize OK!
vf
done in 0.016 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 3.39e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 760          |
| TimeElapsed             | 193          |
| TimestepsSoFar          | 760832       |
| entloss                 | 0.0          |
| entropy                 | 0.17850922   |
| explained_variance_t... | 1.19e-07     |
| meankl                  | 0.011130346  |
| optimgain               | 0.0013826506 |
| surrgain                | 0.0013826506 |
------------------------------------------
********** Iteration 743 ************
Optimizing Policy...
sampling
done in 0.270 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   5.17e-06          0
         1   1.05e-13    0.00969
done in 0.001 seconds
Expected: 0.001 Actual: 0.001
Stepsize OK!
vf
done in 0.017 seconds
-------------------------

********** Iteration 751 ************
Optimizing Policy...
sampling
done in 0.241 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0    0.00105          0
         1   6.14e-13      0.226
done in 0.001 seconds
Expected: 0.012 Actual: 0.010
Stepsize OK!
vf
done in 0.016 seconds
-----------------------------------------
| EpLenMean               | 1e+03       |
| EpRewMean               | 3.41e+03    |
| EpThisIter              | 1           |
| EpisodesSoFar           | 770         |
| TimeElapsed             | 196         |
| TimestepsSoFar          | 770048      |
| entloss                 | 0.0         |
| entropy                 | 0.08802194  |
| explained_variance_t... | -1.19e-07   |
| meankl                  | 0.010088921 |
| optimgain               | 0.009789511 |
| surrgain                | 0.009789511 |
-----------------------------------------
********** Iteration 752 ************
Optimizing Policy...
sampling
done in 0

Stepsize OK!
vf
done in 0.015 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 3.45e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 778          |
| TimeElapsed             | 198          |
| TimestepsSoFar          | 778240       |
| entloss                 | 0.0          |
| entropy                 | 0.052110553  |
| explained_variance_t... | 1.19e-07     |
| meankl                  | 0.007241539  |
| optimgain               | 0.0008766453 |
| surrgain                | 0.0008766453 |
------------------------------------------
********** Iteration 760 ************
Optimizing Policy...
sampling
done in 0.225 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   4.69e-06          0
         1   1.49e-14     0.0313
done in 0.001 seconds
Expected: 0.001 Actual: 0.002
Stepsize OK!
vf
done in 0.016 seconds
-------------------------

********** Iteration 768 ************
Optimizing Policy...
sampling
done in 0.226 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0   0.000248          0
         1   3.15e-13     0.0554
done in 0.001 seconds
Expected: 0.004 Actual: 0.004
Stepsize OK!
vf
done in 0.016 seconds
------------------------------------------
| EpLenMean               | 1e+03        |
| EpRewMean               | 3.46e+03     |
| EpThisIter              | 1            |
| EpisodesSoFar           | 787          |
| TimeElapsed             | 200          |
| TimestepsSoFar          | 787456       |
| entloss                 | 0.0          |
| entropy                 | 0.16443372   |
| explained_variance_t... | 0            |
| meankl                  | 0.00891041   |
| optimgain               | 0.0039037892 |
| surrgain                | 0.0039037892 |
------------------------------------------
********** Iteration 769 ************
Optimizing Policy...
samp

Stepsize OK!
vf
done in 0.016 seconds
-------------------------------------------
| EpLenMean               | 1e+03         |
| EpRewMean               | 3.49e+03      |
| EpThisIter              | 1             |
| EpisodesSoFar           | 795           |
| TimeElapsed             | 202           |
| TimestepsSoFar          | 795648        |
| entloss                 | 0.0           |
| entropy                 | 0.2417669     |
| explained_variance_t... | 1.19e-07      |
| meankl                  | 0.01144153    |
| optimgain               | 0.00029561238 |
| surrgain                | 0.00029561238 |
-------------------------------------------
********** Iteration 777 ************
Optimizing Policy...
sampling
done in 0.227 seconds
computegrad
done in 0.002 seconds
conjugate_gradient
      iter residual norm  soln norm
         0    4.9e-06          0
         1   6.35e-13     0.0066
done in 0.001 seconds
Expected: 0.001 Actual: 0.001
Stepsize OK!
vf
done in 0.015 seconds
-----------

In [ ]:
results_plotter.plot_results([log_dir], time_steps, results_plotter.X_EPISODES, "TRPO")
plt.show()

In [ ]:
obs = env.reset()
for i in range(199):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()